In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from preprocessing.data_loader import prepare_data
import joblib

In [4]:
X_partial, y_partial_orig, X_val, y_val_orig, X_test, y_test_orig = prepare_data("/home/moritz/Data/Task07_Pancreas/",
                                                                              res=256,
                                                                              res_z=48,
                                                                              num_samples=281,
                                                                              mrg_labels=False,
                                                                              label_mode='seg')

... preparing data
... importing training data
... importing labels 
  0%|          | 0/281 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [7]:
#joblib.dump(y_test_orig, "./serialized/data/y_test.lib")
X_partial = joblib.load("./serialized/data/x_partial.lib")
y_partial_orig = joblib.load("./serialized/data/y_partial.lib")
X_val = joblib.load("./serialized/data/x_val.lib")
y_val_orig = joblib.load("./serialized/data/y_val.lib")
X_test = joblib.load("./serialized/data/x_test.lib")
y_test_orig = joblib.load("./serialized/data/y_test.lib")

In [8]:
y_partial = tf.keras.utils.to_categorical(y_partial_orig, num_classes=3)
y_val = tf.keras.utils.to_categorical(y_val_orig, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test_orig, num_classes=3)
y_partial.shape, y_val.shape, y_test.shape

((8448, 256, 256, 3), (960, 256, 256, 3), (4080, 256, 256, 3))

In [9]:
y_partial_tumor = y_partial[..., 2]
y_val_tumor = y_val[..., 2]

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
b_size = 16
def prep(inp): return inp

data_gen_args = dict(
rotation_range=10.,
zoom_range=(0.8, 1.2),
height_shift_range=0.2,
width_shift_range=0.2,
brightness_range=(0.7, 1.2),
preprocessing_function=prep,
rescale=1/255.)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

In [11]:
seed = 42
image_generator = image_datagen.flow(X_partial[..., None], seed=seed, batch_size=b_size)
mask_generator = mask_datagen.flow(y_partial, seed=seed, batch_size=b_size)

train_generator = zip(image_generator, mask_generator)

In [12]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def generalized_dice_loss(y_true, y_pred):
    return 1-dice_coeff(y_true, y_pred)

In [13]:
import segmentation_models as sm
import numpy as np

Segmentation Models: using `tf.keras` framework.


In [14]:
from segmentation_models.metrics import f1_score

def tumor_dice(y_true, y_pred):
    return f1_score(y_true[..., 2], y_pred[..., 2])

def panc_dice(y_true, y_pred):
    return f1_score(y_true[..., 1], y_pred[..., 1])

In [15]:
from models.MoNet import getMoNet
from models.custom_unet import custom_unet
monet = getMoNet(output_classes=3)
unet = custom_unet((256, 256, 1), num_classes=3, filters=64, output_activation='softmax')

In [16]:
base = sm.FPN(input_shape=(256, 256, 3), classes=3, backbone_name='resnet50', encoder_weights='imagenet')
inp = tf.keras.layers.Input(shape=(256, 256, 1))
l1 = tf.keras.layers.Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base(l1)

fpn = tf.keras.Model(inp, out, name=base.name)

In [17]:
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [19]:
monet.compile(loss=dice_loss, optimizer='adam',
              metrics=["accuracy", tumor_dice, panc_dice, dice_coeff, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
fpn.compile(loss=dice_loss, optimizer='adam',
              metrics=["accuracy", tumor_dice, panc_dice, dice_coeff, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [21]:
history = monet.fit(
          train_generator,
          validation_data=(X_val[..., None], y_val),
          steps_per_epoch=X_partial.shape[0]//b_size,
          epochs=90,
          callbacks=[],
          verbose=1)

ss: 0.2677 - accuracy: 0.9976 - tumor_dice: 0.5690 - panc_dice: 0.6774 - dice_coeff: 0.9493 - precision: 0.9983 - recall: 0.9967 - val_loss: 0.3734 - val_accuracy: 0.9973 - val_tumor_dice: 0.1737 - val_panc_dice: 0.7072 - val_dice_coeff: 0.9973 - val_precision: 0.9973 - val_recall: 0.9973
Epoch 37/90
528/528 [==============================] - 114s 216ms/step - loss: 0.2621 - accuracy: 0.9977 - tumor_dice: 0.5764 - panc_dice: 0.6860 - dice_coeff: 0.9502 - precision: 0.9984 - recall: 0.9968 - val_loss: 0.3726 - val_accuracy: 0.9973 - val_tumor_dice: 0.1791 - val_panc_dice: 0.7044 - val_dice_coeff: 0.9973 - val_precision: 0.9973 - val_recall: 0.9973
Epoch 38/90
528/528 [==============================] - 114s 216ms/step - loss: 0.2588 - accuracy: 0.9978 - tumor_dice: 0.5835 - panc_dice: 0.6894 - dice_coeff: 0.9496 - precision: 0.9984 - recall: 0.9968 - val_loss: 0.3712 - val_accuracy: 0.9974 - val_tumor_dice: 0.1973 - val_panc_dice: 0.6905 - val_dice_coeff: 0.9974 - val_precision: 0.9974 -

In [22]:
monet.save_weights("./serialized/weights/monet_panc+tumor_weights.h5")

In [19]:
history = fpn.fit(train_generator,
          validation_data=(X_val[..., None], y_val),
          batch_size=16,
          epochs=80,
          steps_per_epoch=X_partial.shape[0]//16,
          verbose=1)

========================] - 134s 255ms/step - loss: 0.2475 - accuracy: 0.9979 - tumor_dice: 0.5947 - panc_dice: 0.7112 - dice_coeff: 0.9504 - precision_3: 0.9986 - recall_3: 0.9969 - val_loss: 0.3430 - val_accuracy: 0.9968 - val_tumor_dice: 0.4074 - val_panc_dice: 0.5650 - val_dice_coeff: 0.9968 - val_precision_3: 0.9968 - val_recall_3: 0.9968
Epoch 51/200
528/528 [==============================] - 134s 254ms/step - loss: 0.2572 - accuracy: 0.9979 - tumor_dice: 0.5730 - panc_dice: 0.7056 - dice_coeff: 0.9486 - precision_3: 0.9985 - recall_3: 0.9969 - val_loss: 0.4048 - val_accuracy: 0.9961 - val_tumor_dice: 0.3561 - val_panc_dice: 0.4313 - val_dice_coeff: 0.9961 - val_precision_3: 0.9961 - val_recall_3: 0.9961
Epoch 52/200
528/528 [==============================] - 134s 254ms/step - loss: 0.2502 - accuracy: 0.9979 - tumor_dice: 0.5872 - panc_dice: 0.7119 - dice_coeff: 0.9491 - precision_3: 0.9986 - recall_3: 0.9970 - val_loss: 0.2846 - val_accuracy: 0.9973 - val_tumor_dice: 0.4947 - va

KeyboardInterrupt: 

In [20]:
import pandas as pd
h_df = pd.DataFrame(history.history)
plt.plot(h_df)

NameError: name 'history' is not defined

In [26]:
monet.compile(loss=dice_loss, optimizer='adam', metrics=[dice_coeff])
unet.compile(loss=dice_loss, optimizer='adam', metrics=[dice_coeff])

In [27]:
history = monet.fit(
          X_partial[..., None],
          y_partial_tumor[..., None],
          validation_data=(X_val[..., None], y_val_tumor[..., None]),
          steps_per_epoch=X_partial.shape[0]//b_size,
          epochs=200,
          callbacks=[],
          verbose=1)

Epoch 1/200
528/528 [==============================] - 104s 196ms/step - loss: 0.9979 - dice_coeff: 0.0021 - val_loss: 0.9977 - val_dice_coeff: 0.0023
Epoch 2/200
528/528 [==============================] - 104s 197ms/step - loss: 0.9969 - dice_coeff: 0.0031 - val_loss: 0.9964 - val_dice_coeff: 0.0036
Epoch 3/200
528/528 [==============================] - 104s 197ms/step - loss: 0.9953 - dice_coeff: 0.0047 - val_loss: 0.9947 - val_dice_coeff: 0.0053
Epoch 4/200
528/528 [==============================] - 101s 192ms/step - loss: 0.9923 - dice_coeff: 0.0077 - val_loss: 0.9930 - val_dice_coeff: 0.0070
Epoch 5/200
528/528 [==============================] - 100s 190ms/step - loss: 0.9876 - dice_coeff: 0.0124 - val_loss: 0.9901 - val_dice_coeff: 0.0099
Epoch 6/200
528/528 [==============================] - 100s 190ms/step - loss: 0.9796 - dice_coeff: 0.0204 - val_loss: 0.9840 - val_dice_coeff: 0.0160
Epoch 7/200
528/528 [==============================] - 100s 190ms/step - loss: 0.9668 - dice_c

KeyboardInterrupt: 

In [204]:
y_partial_tumor.shape

(8448, 256, 256)

In [28]:
fpn.evaluate(X_test[..., None], y_test, verbose=1)

128/128 [==============================] - 13s 99ms/step - loss: 0.6052 - accuracy: 0.9970 - tumor_dice: 0.2599 - panc_dice: 0.6444 - precision_7: 0.9961 - recall_7: 0.9976


[0.6052348017692566,
 0.9969928860664368,
 0.2599358558654785,
 0.644449770450592,
 0.9961434602737427,
 0.9976320266723633]

In [22]:
monet.evaluate(X_test[..., None], y_test, verbose=1)

128/128 [==============================] - 15s 118ms/step - loss: 0.0030 - accuracy: 0.9970 - tumor_dice: 0.1973 - panc_dice: 0.6296 - precision: 0.9970 - recall: 0.9970


[0.0029794713482260704,
 0.9970209002494812,
 0.19725310802459717,
 0.629561722278595,
 0.9970208406448364,
 0.9970208406448364]

In [86]:
y_pred = monet.predict(X_partial[..., None], batch_size=16)

In [ ]:
for i in range(10,20):
    print("pred")
    plt.imshow(y_pred[i, ..., 1])
    plt.show()
    print("actual")
    plt.imshow(y_partial[i, ...,1])
    plt.show()